In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
import schedulers

import tensorflow as tf
assert tf.config.list_physical_devices('GPU')

In [2]:
BATCH_SIZE = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(UNITS=500):
    inp = keras.Input(shape=(None,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, UNITS, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')(embed)
    layer = layer + layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = embed + layers.Dense(UNITS, activation='relu')(layer)

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=inp, outputs=outputs)

    return model

model = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 500)    22000       input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, None, 500)    4004000     embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, None, 500)    4004000     bidirectional[0][0]              
______________________________________________________________________________________________

In [49]:

# masked version of accuracy and sce
def accuracy(real, pred):
    acc = tf.keras.metrics.sparse_categorical_accuracy(real, pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(real, 0)), dtype=acc.dtype)
    acc *= mask

    return tf.reduce_sum(acc) / tf.reduce_sum(mask)

def sparse_categorical_crossentropy(y_true, y_pred, sample_weight=None):
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(y_true, 0)), dtype=loss.dtype)
    loss *= mask

    return tf.reduce_sum(loss) / tf.reduce_sum(mask) 

def fit(train_validation, scheduler=None, verbose=1, lr=1e-4, epochs=1):
    train, valid = train_validation
    
    model.compile(loss=sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})
    callbacks = []
    if isinstance(scheduler, schedulers.CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)    
    
    x  = train.normalized
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin }
    
    if valid is None:
        return model.fit(x, y, batch_size=BATCH_SIZE, epochs=epochs, verbose=verbose, callbacks=callbacks)
    
    vx = valid.normalized
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin }
    
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=epochs, verbose=verbose, callbacks=callbacks)


MAXLEN = 64
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    
    if validation == 0:
        corpus = dataset.read_corpus(filenames, maxlen)
        train = dataset.Data.concatenate(corpus)
        train.shuffle()
        return train, None
    
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid


def load_test_data(maxlen=MAXLEN):
    return dataset.load_file('test/ModernTestCorpus-HebrewWiki1-simple.txt', maxlen) 


In [21]:
data_mix = load_data(['poetry', 'rabanit', 'pre_modern'], validation=0)

In [22]:
data_modern = load_data(['modern'], validation=0)

In [25]:
model.load_weights('./checkpoints/uninit')
history = fit(data_mix, scheduler=schedulers.CircularLearningRate(30e-4, 80e-4, 1e-4), epochs=1)
model.save_weights('./checkpoints/mix')

2426/2426 [==============================] - 244s 101ms/step - loss: 0.2656 - N_loss: 0.2032 - D_loss: 0.0554 - S_loss: 0.0070 - N_accuracy: 0.9326 - D_accuracy: 0.9791 - S_accuracy: 0.99805s - loss: 0.2800 - N_loss: 0.2146 - D_loss: 0.0580 - S_loss: 0.007 - ETA: 15s - loss: 0.2 - ETA: 9s - loss: 0.2704 - N_loss: 0.2070 - D_l


In [26]:
model.load_weights('./checkpoints/mix')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(50e-4, 50e-4, 1e-5))
model.save_weights('./checkpoints/modern')

292/292 [==============================] - 29s 101ms/step - loss: 0.1518 - N_loss: 0.1134 - D_loss: 0.0361 - S_loss: 0.0023 - N_accuracy: 0.9639 - D_accuracy: 0.9873 - S_accuracy: 0.9995


In [27]:
model.load_weights('./checkpoints/modern')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-4, 15e-4, 1e-5), epochs=1)
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-5, 1e-4, 1e-5), epochs=1)
model.save_weights('./checkpoints/modern_over')

292/292 [==============================] - 29s 101ms/step - loss: 0.0299 - N_loss: 0.0205 - D_loss: 0.0090 - S_loss: 4.2268e-04 - N_accuracy: 0.9937 - D_accuracy: 0.9969 - S_accuracy: 0.99993s - loss: 0.0300 - N_loss: 0.0205 - D_loss: 0.0091 - S_loss: 4.1084e-04 - N_accuracy: 0.9937 - D_accu


In [ ]:
model.load_weights('./checkpoints/modern')
model.save('modern.h5')

In [ ]:
import tensorflowjs as tfjs
model.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(model, '.')

In [46]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, s):
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], ts=batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(data.text[s], ts=batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    total = []
    for i, (b, a, e) in enumerate(zip(batch, actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        res = expected_niqqud[i][b > 16] == actual_niqqud[i][b > 16]
        total.extend(res)
        print(round(np.mean(res), 2), f'({np.sum(res)} out of {len(res)})')
        print()
    print(round(np.mean(total), 3))

print_predictions(test, slice(0, None))

מצוי:  "דֶּלֶק" חֶבְרַת הַדֶּלֶק הַיִּשְׂרְאֵלִית "דֶּלֶק" חֶבְרַת הַדֶּלֶק הַיִּשְׂרְאֵלִית בע"מ, הַיְּדוּעָה 
רצוי:  "דֶּלֶק" חֶבְרַת הַדֶּלֶק הַיִּשְׂרְאֵלִית "דֶּלֶק" חֶבְרַת הַדֶּלֶק הַיִּשְׂרְאֵלִית בע"מ, הַיְּדוּעָה 
1.0 (47 out of 47)

מצוי:  יוֹתֵר בְּשֵׁם דֶּלֶק, הִיא אַחַת מֵאַרְבַּע חֲבְרוֹת הַדֶּלֶק הַגְּדוֹלוֹת בְּיִשְׂרָאֵל. בְּבַעֲלוּתָהּ 
רצוי:  יוֹתֵר בַּשֵּׁם דֶּלֶק, הִיא אַחַת מֵאַרְבַּע חֶבְרוֹת הַדֶּלֶק הַגְּדוֹלוֹת בְּיִשְׂרָאֵל. בְּבַעֲלוּתָהּ 
0.96 (48 out of 50)

מצוי:  מֵאוֹת תַּחֲנוֹת דֶּלֶק בְּרַחֲבֵי הָאָרֶץ, בְּחֶלְקָן גַּם חֲנוּיוֹת נוֹחוּת תַּחַת הַמּוּתָג 
רצוי:  מְאוֹת תַּחֲנוֹת דֶּלֶק בְּרַחֲבֵי הָאָרֶץ, בְּחֶלְקָן גַּם חֲנוּיוֹת נוֹחוּת תַּחַת הַמּוּתָג 
0.98 (46 out of 47)

מצוי:  "מַנְטָה" וּמִתְחָמִּים קִמְעוּנָאִיִּים. בְּבַּעֲלוּת הַחֶבְרָה מִסְפַּר חֶבְרוֹת-בָּנוֹת בִּתְחוּמֵי 
רצוי:  "מֶנְטָה" וּמִתְחָמִים קִמְעוֹנָאִיִּים. בְּבַעֲלוּת הַחֶבְרָה מִסְפַּר חֲבָרוֹת-בָּנוֹת בִּתְחוּמֵי 
0.92 (46 out of 50)

מצוי:  הָאֵנֶרְגִּיָּה וְהַלּוֹגִיסְטִ

In [ ]:
for i in range(50):
    model.load_weights('./checkpoints/pre_modern')
    p1 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    p2 = np.exp(np.random.uniform(low=np.log(1e-4), high=np.log(1e-1)))
    p3 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    print(p1, p2, p3, end=', ', sep=', ')
    history = fit(data_modern, scheduler=schedulers.CircularLearningRate(p1, p2, p3), verbose=0)
    print(history.history['val_N_accuracy'][0])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=3)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['N', 'D', 'S'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:

import dataset

In [17]:
test = load_test_data()

In [51]:
x  = test.normalized
y  = {'N': test.niqqud, 'D': test.dagesh, 'S': test.sin }

model.compile(loss=sparse_categorical_crossentropy,
              metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})

model.evaluate(x=x, y=y, batch_size=BATCH_SIZE)

10/10 [==============================] - 0s 38ms/step - loss: 0.2172 - N_loss: 0.1756 - D_loss: 0.0389 - S_loss: 0.0026 - N_accuracy: 0.9482 - D_accuracy: 0.9875 - S_accuracy: 0.9994


[0.2171633541584015,
 0.1755978763103485,
 0.03893987834453583,
 0.0026256279088556767,
 0.9481841325759888,
 0.9875186681747437,
 0.9993783831596375]